In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, Scale
from PIL import Image, ImageTk, ImageOps, ImageFilter, ImageEnhance, ImageChops, ImageDraw, ImageFont
import numpy as np
import cv2
import math

class DockableWindow(tk.Toplevel):
    """A window that can be docked to the main window or float independently."""
    def __init__(self, parent, title="Panel", initial_position=None, dock_side=tk.RIGHT):
        super().__init__(parent)
        self.parent = parent
        self.title(title)
        self.transient(parent)  # Make window a child of parent
        
        # Set initial position if provided
        if initial_position:
            self.geometry(f"+{initial_position[0]}+{initial_position[1]}")
        
        # Create a frame to hold the content
        self.content_frame = ttk.Frame(self)
        self.content_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Initialize state as floating
        self.is_docked = False
        self.dock_side = dock_side
        self.dock_frame = None
        
        # Create a title bar
        self.title_bar = ttk.Frame(self, relief=tk.RAISED, padding=2)
        self.title_bar.pack(fill=tk.X, side=tk.TOP)
        
        # Title label
        ttk.Label(self.title_bar, text=title).pack(side=tk.LEFT, padx=5)
        
        # Dock/Undock button
        self.dock_button = ttk.Button(self.title_bar, text="Dock", command=self.toggle_dock)
        self.dock_button.pack(side=tk.RIGHT, padx=2)
        
        # Make the window draggable
        self.title_bar.bind("<ButtonPress-1>", self.start_move)
        self.title_bar.bind("<ButtonRelease-1>", self.stop_move)
        self.title_bar.bind("<B1-Motion>", self.do_move)
        
        # Add window management
        self.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Variables for window dragging
        self.x = 0
        self.y = 0
        
    def start_move(self, event):
        self.x = event.x
        self.y = event.y
    
    def stop_move(self, event):
        self.x = None
        self.y = None
    
    def do_move(self, event):
        if self.is_docked:
            return  # Don't move if docked
        
        deltax = event.x - self.x
        deltay = event.y - self.y
        x = self.winfo_x() + deltax
        y = self.winfo_y() + deltay
        self.geometry(f"+{x}+{y}")
    
    def toggle_dock(self):
        if self.is_docked:
            self.undock()
        else:
            self.dock()
    
    def dock(self):
        # Create a frame in the parent to hold this panel
        if not self.dock_frame:
            self.dock_frame = ttk.Frame(self.parent)
            
        # Move the content to the dock frame
        self.content_frame.pack_forget()
        self.content_frame = ttk.Frame(self.dock_frame)
        self.content_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Use the parent's pack method to position this frame
        self.dock_frame.pack(side=self.dock_side, fill=tk.Y, pady=5, padx=5)
        
        # Hide the floating window
        self.withdraw()
        self.is_docked = True
        self.dock_button.config(text="Float")
        
        # Re-create the content in the docked frame
        self.create_content(self.content_frame)
    
    def undock(self):
        # Save the current size
        width = self.content_frame.winfo_width()
        height = self.content_frame.winfo_height()
        
        # Remove the frame from the parent
        if self.dock_frame:
            self.dock_frame.pack_forget()
        
        # Move content back to the floating window
        self.content_frame.pack_forget()
        self.content_frame = ttk.Frame(self)
        self.content_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Show the window and position it sensibly
        parent_x = self.parent.winfo_rootx()
        parent_y = self.parent.winfo_rooty()
        parent_width = self.parent.winfo_width()
        
        if self.dock_side == tk.RIGHT:
            x = parent_x + parent_width + 10
            y = parent_y + 50
        else:
            x = parent_x + 50
            y = parent_y + 50
        
        self.geometry(f"{width}x{height}+{x}+{y}")
        self.deiconify()
        self.is_docked = False
        self.dock_button.config(text="Dock")
        
        # Re-create the content in the undocked frame
        self.create_content(self.content_frame)
    
    def on_close(self):
        # Instead of destroying, just hide and dock
        self.withdraw()
        if not self.is_docked:
            self.dock()
    
    def create_content(self, parent):
        """Override this method to create the panel's content"""
        pass

class StyleSelectorPanel(DockableWindow):
    """Panel for selecting styles and common controls"""
    def __init__(self, parent, app, **kwargs):
        super().__init__(parent, title="Style Selection", **kwargs)
        self.app = app
        self.create_content(self.content_frame)
    
    def create_content(self, parent):
        # Clear existing content
        for widget in parent.winfo_children():
            widget.destroy()
        
        # Style options
        styles = [
            "None", 
            "Style 1: Abstract Portrait", 
            "Style 2: Colorful Blur",
            "Style 3: Mosaic Effect",
            "Style 4: Sabattier Effect"
        ]
        
        # Create the style selection frame
        style_frame = ttk.LabelFrame(parent, text="Select Style")
        style_frame.pack(fill=tk.X, padx=5, pady=5)
        
        for style in styles:
            ttk.Radiobutton(
                style_frame, 
                text=style, 
                variable=self.app.style_var, 
                value=style, 
                command=self.app.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)
        
        # Apply button
        ttk.Button(
            style_frame, 
            text="Apply Style", 
            command=self.app.apply_style
        ).pack(fill=tk.X, padx=5, pady=10)
        
        # Reset button
        ttk.Button(
            style_frame, 
            text="Reset to Original", 
            command=self.app.reset_image
        ).pack(fill=tk.X, padx=5, pady=5)
        
        # Common parameters frame
        common_frame = ttk.LabelFrame(parent, text="Common Parameters")
        common_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Create common sliders
        labels = ["Contrast", "Brightness", "Saturation"]
        params = ["contrast", "brightness", "saturation"]
        defaults = [100, 100, 100]
        
        for i, (label, param, default) in enumerate(zip(labels, params, defaults)):
            frame = ttk.Frame(common_frame)
            frame.pack(fill=tk.X, padx=5, pady=5)
            
            ttk.Label(frame, text=label).pack(anchor=tk.W)
            
            self.app.param_controls[param] = tk.IntVar(value=default)
            slider = Scale(
                frame, 
                from_=0, 
                to=200, 
                orient=tk.HORIZONTAL, 
                variable=self.app.param_controls[param],
                command=lambda _: self.app.apply_style()
            )
            slider.pack(fill=tk.X)

class ParameterPanel(DockableWindow):
    """Panel for style-specific parameters"""
    def __init__(self, parent, app, **kwargs):
        super().__init__(parent, title="Style Parameters", **kwargs)
        self.app = app
        
        # Create a canvas with scrollbar for parameters
        self.canvas_frame = ttk.Frame(self.content_frame)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame)
        self.scrollbar = ttk.Scrollbar(self.canvas_frame, orient="vertical", command=self.canvas.yview)
        
        # Parameter frame inside canvas
        self.param_frame = ttk.Frame(self.canvas)
        
        # Configure scrolling
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Create window inside canvas
        self.window_id = self.canvas.create_window((0, 0), window=self.param_frame, anchor="nw")
        
        # Configure canvas to expand with frame and update scrollregion
        def on_frame_configure(event):
            self.canvas.configure(scrollregion=self.canvas.bbox("all"))
            # Update the width of the window to match the canvas width
            self.canvas.itemconfig(self.window_id, width=self.canvas.winfo_width())
        
        def on_canvas_configure(event):
            # Update the width of the window to match the canvas width
            self.canvas.itemconfig(self.window_id, width=self.canvas.winfo_width())
        
        self.param_frame.bind("<Configure>", on_frame_configure)
        self.canvas.bind("<Configure>", on_canvas_configure)
        
        # Mouse wheel scrolling
        def _on_mousewheel(event):
            self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        
        self.canvas.bind_all("<MouseWheel>", _on_mousewheel)
        
        # Set the app's param_frame to this one
        self.app.param_frame = self.param_frame
    
    def create_content(self, parent):
        # The content is created dynamically when styles are selected
        # through initialize_parameters, no need to do anything here
        pass

class ImageStyleTransformer:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Style Transformer")
        self.root.geometry("1200x800")
        
        # Initialize variables
        self.original_image = None
        self.displayed_image = None
        self.processed_image = None
        self.current_style = "None"

        self.style_var = tk.StringVar(value="None")  # Add this line

        # Initialize the param_controls dictionary
        self.param_controls = {}

        # Style descriptions for status bar
        self.style_descriptions = {
            "None": "Original image without effects",
            "Style 1: Datamosh": "Datamosh effect with P-frame/I-frame corruption and displacement mapping - high contrast black and white",
            "Style 2: Colorful Blur": "Vibrant color shifts with blur effects",
            "Style 3: Mosaic Effect": "Fractal mirror and slice distortion effects",
            "Style 4: Sabattier Effect": " ",
            "Style 5: ASCII Art": "Transform image into patterns of ASCII characters based on brightness levels"

        }

        # Mask variables for the colorful blur style
        self.mask_tool_mode = "brush"  # "brush" or "lasso"
        self.lasso_points = []  # To store points for the lasso tool
        self.temp_lasso_line = None  # For visualization while drawing lasso

        self.masks = []  # List to store mask objects
        self.drawing = False
        self.mask_mode = False
        self.mask_radius = 30
        self.mask_feather = 20
        self.mask_composite = None  # Composite of all masks

        # Create main frames
        self.create_frames()
        
        # Create widgets
        self.create_menu_bar()
        self.create_image_display()
        self.create_style_selector()
        self.create_parameter_controls()
        
        # Initialize parameters
        self.initialize_parameters()
        
    def create_frames(self):
        # Left frame for image display
        self.left_frame = ttk.Frame(self.root)
        self.left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Right frame for controls
        self.right_frame = ttk.Frame(self.root, width=350)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)
        self.right_frame.pack_propagate(False)
        
    def create_menu_bar(self):
        menu_bar = tk.Menu(self.root)
        
        # File menu
        file_menu = tk.Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open Image", command=self.open_image)
        file_menu.add_command(label="Save Image", command=self.save_image)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)
        
        menu_bar.add_cascade(label="File", menu=file_menu)
        self.root.config(menu=menu_bar)
        
    def create_image_display(self):
        # Canvas for image display
        self.canvas_frame = ttk.Frame(self.left_frame)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg="lightgray")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Bind mouse events for mask drawing
        self.canvas.bind("<ButtonPress-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)
        
        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("No image loaded. Please open an image.")
        self.status_bar = ttk.Label(self.left_frame, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(fill=tk.X, pady=(5, 0))
        
    def create_style_selector(self):
        # Style selection frame
        style_frame = ttk.LabelFrame(self.right_frame, text="Select Style")
        style_frame.pack(fill=tk.X, padx=5, pady=10)

        # Style options
        styles = [
            "None",
            "Style 1: Datamosh",
            "Style 2: Colorful Blur",
            "Style 3: Mosaic Effect",
            "Style 4: PixiDigi Sabattier",
            "Style 5: ASCII"

        ]

        self.style_var = tk.StringVar()
        self.style_var.set(styles[0])

        for style in styles:
            ttk.Radiobutton(
                style_frame,
                text=style,
                variable=self.style_var,
                value=style,
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)

        # Apply button
        ttk.Button(
            style_frame,
            text="Apply Style",
            command=self.apply_style
        ).pack(fill=tk.X, padx=5, pady=10)

        # Reset button
        ttk.Button(
            style_frame,
            text="Reset to Original",
            command=self.reset_image
        ).pack(fill=tk.X, padx=5, pady=5)


    def create_parameter_controls(self):
        # Parameter controls frame
        self.param_frame = ttk.LabelFrame(self.right_frame, text="Style Parameters")
        self.param_frame.pack(fill=tk.X, padx=5, pady=10)

        # We'll populate this with specific controls when a style is selected
        self.param_controls = {}
    
    def initialize_parameters(self):
        """Initialize style parameters based on selected style."""
        # Clear existing controls
        for widget in self.param_frame.winfo_children():
            widget.destroy()
        
        # Clean up any previous mousewheel bindings
        if hasattr(self, 'sabattier_canvas'):
            try:
                self.sabattier_canvas.unbind_all("<MouseWheel>")
            except:
                pass
        
        # Create common parameters
        self.param_controls = {
            "contrast": self.create_slider("Contrast", 0, 200, 100),
            "brightness": self.create_slider("Brightness", 0, 200, 100),
            "saturation": self.create_slider("Saturation", 0, 200, 100),
        }
        
        # Add style-specific parameters based on selected style
        style = self.style_var.get()
        
        if "Style 1" in style:  # Abstract Portrait / Datamosh
            self.param_controls["pixelate"] = self.create_slider("Block Size", 1, 20, 5)
            self.param_controls["block_variety"] = self.create_slider("Block Size Variety", 0, 100, 50)  # Increased default
            self.param_controls["glitch_amount"] = self.create_slider("Displacement Amount", 0, 100, 70)
            self.param_controls["artifact_amount"] = self.create_slider("Glitch Artifacts", 0, 100, 30)  # New parameter
            self.param_controls["blur"] = self.create_slider("Blur", 0, 100, 0)
            self.param_controls["motion_blur"] = self.create_slider("Motion Blur", 0, 100, 0)
            self.param_controls["random_seed"] = self.create_slider("Random Seed", 0, 1000, 42)
            
            # Add presets for different looks
            presets_frame = ttk.LabelFrame(self.param_frame, text="Presets")
            presets_frame.pack(fill=tk.X, padx=5, pady=5)
            
            preset_values = [
                ("Subtle Glitch", 3, 30, 50, 20, 10, 0, 0, 42),
                ("Heavy Corruption", 7, 70, 90, 10, 70, 0, 0, 123),
                ("Scan Lines", 4, 40, 60, 30, 50, 0, 20, 256),
                ("Blocky Portrait", 8, 60, 40, 50, 20, 0, 0, 789)
            ]
            
            self.abstract_preset_var = tk.StringVar(value="Custom")
            preset_combo = ttk.Combobox(
                presets_frame,
                textvariable=self.abstract_preset_var,
                values=[preset[0] for preset in preset_values],
                state="readonly"
            )
            preset_combo.pack(fill=tk.X, pady=5)
            
            # Function to apply preset
            def apply_abstract_preset(event):
                preset_name = self.abstract_preset_var.get()
                for preset in preset_values:
                    if preset[0] == preset_name:
                        self.param_controls["pixelate"].set(preset[1])
                        self.param_controls["block_variety"].set(preset[2])
                        self.param_controls["glitch_amount"].set(preset[3])
                        self.param_controls["artifact_amount"].set(preset[5])
                        self.param_controls["blur"].set(preset[6])
                        self.param_controls["motion_blur"].set(preset[7])
                        self.param_controls["random_seed"].set(preset[8])
                        self.apply_style()
                        break
            
            preset_combo.bind("<<ComboboxSelected>>", apply_abstract_preset)
            
            # Add randomize button
            ttk.Button(
                presets_frame,
                text="Randomize Seed",
                command=lambda: (self.param_controls["random_seed"].set(np.random.randint(0, 1000)), self.apply_style())
            ).pack(fill=tk.X, padx=5, pady=5)
            
        elif "Style 2" in style:  # Colorful Blur
            self.param_controls["blur_radius"] = self.create_slider("Blur Radius", 0, 50, 15)
            self.param_controls["color_shift"] = self.create_slider("Color Shift", 0, 100, 50)
            
            # Add mask mode controls
            mask_frame = ttk.Frame(self.param_frame)
            mask_frame.pack(fill=tk.X, padx=5, pady=5)
            
            tools_frame = ttk.Frame(self.param_frame)
            tools_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(tools_frame, text="Mask Tool:").pack(anchor=tk.W)

            self.mask_tool_var = tk.StringVar(value="brush")
            ttk.Radiobutton(
                tools_frame,
                text="Brush Tool",
                variable=self.mask_tool_var,
                value="brush",
                command=self.change_mask_tool
            ).pack(anchor=tk.W, padx=5, pady=2)

            ttk.Radiobutton(
                tools_frame,
                text="Lasso Tool",
                variable=self.mask_tool_var,
                value="lasso",
                command=self.change_mask_tool
            ).pack(anchor=tk.W, padx=5, pady=2)

            # Mask checkbox
            self.mask_var = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                mask_frame,
                text="Enable Mask Drawing",
                variable=self.mask_var,
                command=self.toggle_mask_mode
            ).pack(anchor=tk.W, pady=2)
            
            # Mask radius slider
            mask_radius_frame = ttk.Frame(self.param_frame)
            mask_radius_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(mask_radius_frame, text="Mask Brush Size:").pack(anchor=tk.W)
            
            self.mask_radius_var = tk.IntVar(value=30)
            Scale(
                mask_radius_frame,
                from_=5,
                to=100,
                orient=tk.HORIZONTAL,
                variable=self.mask_radius_var
            ).pack(fill=tk.X)
            
            # Mask feather slider
            mask_feather_frame = ttk.Frame(self.param_frame)
            mask_feather_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(mask_feather_frame, text="Mask Feather Amount:").pack(anchor=tk.W)
            
            self.mask_feather_var = tk.IntVar(value=20)
            Scale(
                mask_feather_frame,
                from_=0,
                to=50,
                orient=tk.HORIZONTAL,
                variable=self.mask_feather_var
            ).pack(fill=tk.X)
            
            # Clear masks button
            ttk.Button(
                self.param_frame,
                text="Clear All Masks",
                command=self.clear_masks
            ).pack(fill=tk.X, padx=5, pady=10)
            
        elif "Style 3" in style:  # Mosaic Effect
            mosaic_type = getattr(self, "mosaic_type_var", tk.StringVar(value="flip")).get()
            
            # Mosaic type dropdown
            mosaic_frame = ttk.Frame(self.param_frame)
            mosaic_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(mosaic_frame, text="Mosaic Type:").pack(anchor=tk.W)
            
            self.mosaic_type_var = tk.StringVar(value=mosaic_type)
            mosaic_combo = ttk.Combobox(
                mosaic_frame,
                textvariable=self.mosaic_type_var,
                values=["flip", "slice"],
                state="readonly"
            )
            mosaic_combo.pack(fill=tk.X, pady=5)
            mosaic_combo.bind("<<ComboboxSelected>>", lambda e: (self.initialize_parameters(), self.apply_style()))

            # Common mosaic parameters
            self.param_controls["units"] = self.create_slider("Number of Units", 2, 120, 20)
            
            # Pattern type dropdown (for both flip and slice)
            pattern_frame = ttk.Frame(self.param_frame)
            pattern_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(pattern_frame, text="Pattern:").pack(anchor=tk.W)
            
            default_pattern = getattr(self, "pattern_type_var", tk.StringVar(value="grid")).get()
            self.pattern_type_var = tk.StringVar(value=default_pattern)
            pattern_combo = ttk.Combobox(
                pattern_frame,
                textvariable=self.pattern_type_var,
                values=["vertical", "horizontal", "grid"],
                state="readonly"
            )
            pattern_combo.pack(fill=tk.X, pady=5)
            pattern_combo.bind("<<ComboboxSelected>>", lambda e: self.apply_style())

            # Feathered transitions as a checkbox
            feather_frame = ttk.Frame(self.param_frame)
            feather_frame.pack(fill=tk.X, padx=5, pady=5)
            self.feather_var = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                feather_frame, 
                text="Feathered Transitions", 
                variable=self.feather_var,
                command=self.apply_style
            ).pack(anchor=tk.W)
            
            if mosaic_type == "slice":
                # Slice-specific parameters
                self.param_controls["edge_intensity"] = self.create_slider("Edge Intensity", 0, 200, 100)
                self.param_controls["x_shift"] = self.create_slider("X Shift", -100, 100, 0)
                self.param_controls["y_shift"] = self.create_slider("Y Shift", -100, 100, 30)
                self.param_controls["scale_shift"] = self.create_slider("Scale Shift", -200, 200, 0)

        elif "Style 4" in style:  # Sabattier Effect
            # Set up a scrollable frame for Sabattier Effect parameters
            
            # Make sure param_frame has a fixed height to allow scrolling
            self.param_frame.configure(height=400)  # Adjust height as needed
            
            # Create canvas and scrollbar
            canvas = tk.Canvas(self.param_frame, borderwidth=0, highlightthickness=0)
            scrollbar = ttk.Scrollbar(self.param_frame, orient=tk.VERTICAL, command=canvas.yview)
            
            # Configure canvas
            canvas.configure(yscrollcommand=scrollbar.set)
            
            # Pack scrollbar and canvas
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
            canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            
            # Create a frame inside the canvas to hold all controls
            scrollable_frame = ttk.Frame(canvas)
            
            # Add scrollable frame to canvas
            canvas_window = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
            
            # Make scrollable frame width match canvas width
            def update_scrollregion(event):
                canvas.configure(scrollregion=canvas.bbox("all"))
            
            def update_frame_width(event):
                canvas.itemconfig(canvas_window, width=canvas.winfo_width())
            
            scrollable_frame.bind("<Configure>", update_scrollregion)
            canvas.bind("<Configure>", update_frame_width)
            
            # Add mousewheel scrolling
            def on_mousewheel(event):
                canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
            
            canvas.bind_all("<MouseWheel>", on_mousewheel)
            
            # Store canvas for cleanup
            self.sabattier_canvas = canvas
            
            # Helper function to create sliders in the scrollable frame
            def create_sabattier_slider(label, min_val, max_val, default_val):
                frame = ttk.Frame(scrollable_frame)
                frame.pack(fill=tk.X, padx=5, pady=5)
                
                ttk.Label(frame, text=label).pack(anchor=tk.W)
                
                var = tk.IntVar(value=default_val)
                slider = Scale(
                    frame,
                    from_=min_val,
                    to=max_val,
                    orient=tk.HORIZONTAL,
                    variable=var,
                    command=lambda _: self.apply_style()
                )
                slider.pack(fill=tk.X)
                
                return var
            
            # Create the actual parameters
            self.param_controls["threshold"] = create_sabattier_slider("Threshold", 0, 255, 100)
            self.param_controls["edge_strength"] = create_sabattier_slider("Edge Effect", 0, 100, 30)
            self.param_controls["contrast"] = create_sabattier_slider("Sabattier Contrast", 50, 200, 130)
            self.param_controls["brightness"] = create_sabattier_slider("Sabattier Brightness", 50, 150, 100)
            self.param_controls["developer_variation"] = create_sabattier_slider("Developer Variation", 0, 50, 0)
            self.param_controls["grain_amount"] = create_sabattier_slider("Film Grain", 0, 50, 0)
            
            # Inversion controls
            inversion_frame = ttk.LabelFrame(scrollable_frame, text="Selective Inversion")
            inversion_frame.pack(fill=tk.X, padx=5, pady=5)
            
            self.param_controls["invert_shadows"] = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                inversion_frame,
                text="Invert Shadows",
                variable=self.param_controls["invert_shadows"],
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)
            
            self.param_controls["invert_midtones"] = tk.BooleanVar(value=True)
            ttk.Checkbutton(
                inversion_frame,
                text="Invert Midtones",
                variable=self.param_controls["invert_midtones"],
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)
            
            self.param_controls["invert_highlights"] = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                inversion_frame,
                text="Invert Highlights",
                variable=self.param_controls["invert_highlights"],
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)
            
            # Threshold sliders
            self.param_controls["shadow_threshold"] = create_sabattier_slider("Shadow Threshold", 0, 128, 64)
            self.param_controls["highlight_threshold"] = create_sabattier_slider("Highlight Threshold", 128, 255, 192)
            
            # Tonal range frame
            tonal_frame = ttk.LabelFrame(scrollable_frame, text="Output Tonal Range")
            tonal_frame.pack(fill=tk.X, padx=5, pady=5)

            # Min slider with fixed-width value display
            self.param_controls["tonal_min"] = tk.IntVar(value=0)
            min_slider_frame = ttk.Frame(tonal_frame)
            min_slider_frame.pack(fill=tk.X, padx=5, pady=2)

            # Create a fixed-width label for the value
            min_value_label = ttk.Label(min_slider_frame, width=3, anchor=tk.E)
            min_value_var = tk.StringVar(value="0")  # Use string var for formatting

            # Update function that formats the value
            def update_min_label(*args):
                min_value_var.set(f"{self.param_controls['tonal_min'].get():3d}")
                self.apply_style()

            self.param_controls["tonal_min"].trace_add("write", update_min_label)
            min_value_label.configure(textvariable=min_value_var)

            ttk.Label(min_slider_frame, text="Min:").pack(side=tk.LEFT, padx=(0, 10))
            ttk.Scale(
                min_slider_frame,
                from_=0,
                to=255,
                orient=tk.HORIZONTAL,
                variable=self.param_controls["tonal_min"],
                command=lambda _: update_min_label()
            ).pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
            min_value_label.pack(side=tk.RIGHT, padx=(0, 5))

            # Max slider with fixed-width value display
            self.param_controls["tonal_max"] = tk.IntVar(value=150)
            max_slider_frame = ttk.Frame(tonal_frame)
            max_slider_frame.pack(fill=tk.X, padx=5, pady=2)

            # Create a fixed-width label for the value
            max_value_label = ttk.Label(max_slider_frame, width=3, anchor=tk.E)
            max_value_var = tk.StringVar(value="150")  # Use string var for formatting

            # Update function that formats the value
            def update_max_label(*args):
                max_value_var.set(f"{self.param_controls['tonal_max'].get():3d}")
                self.apply_style()

            self.param_controls["tonal_max"].trace_add("write", update_max_label)
            max_value_label.configure(textvariable=max_value_var)

            ttk.Label(max_slider_frame, text="Max:").pack(side=tk.LEFT, padx=(0, 10))
            ttk.Scale(
                max_slider_frame,
                from_=0,
                to=255,
                orient=tk.HORIZONTAL,
                variable=self.param_controls["tonal_max"],
                command=lambda _: update_max_label()
            ).pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
            max_value_label.pack(side=tk.RIGHT, padx=(0, 5))

            
            # Invert tonal selection
            self.param_controls["invert_tonal"] = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                tonal_frame,
                text="Invert Selection",
                variable=self.param_controls["invert_tonal"],
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=5)
            
            # Presets
            presets_frame = ttk.LabelFrame(scrollable_frame, text="Presets")
            presets_frame.pack(fill=tk.X, padx=5, pady=5)
            
            preset_values = [
                ("Man Ray Portrait", 100, 30, 140, 110, 10, 10, False, True, False, 64, 192),
                ("High Contrast", 128, 50, 180, 90, 5, 0, False, True, False, 50, 200),
                ("Subtle Effect", 90, 20, 120, 100, 0, 5, True, False, False, 80, 180),
                ("Dramatic Edges", 110, 80, 150, 100, 15, 15, False, True, True, 60, 210)
            ]
            
            self.preset_var = tk.StringVar(value="Custom")
            preset_combo = ttk.Combobox(
                presets_frame,
                textvariable=self.preset_var,
                values=[preset[0] for preset in preset_values],
                state="readonly"
            )
            preset_combo.pack(fill=tk.X, pady=5)
            
            # Apply preset function
            def apply_preset(event):
                preset_name = self.preset_var.get()
                for preset in preset_values:
                    if preset[0] == preset_name and len(preset) >= 12:
                        self.param_controls["threshold"].set(preset[1])
                        self.param_controls["edge_strength"].set(preset[2])
                        self.param_controls["contrast"].set(preset[3])
                        self.param_controls["brightness"].set(preset[4])
                        self.param_controls["developer_variation"].set(preset[5])
                        self.param_controls["grain_amount"].set(preset[6])
                        self.param_controls["invert_shadows"].set(preset[7])
                        self.param_controls["invert_midtones"].set(preset[8])
                        self.param_controls["invert_highlights"].set(preset[9])
                        self.param_controls["shadow_threshold"].set(preset[10])
                        self.param_controls["highlight_threshold"].set(preset[11])
                        # Reset tonal range
                        self.param_controls["tonal_min"].set(0)
                        self.param_controls["tonal_max"].set(255)
                        self.param_controls["invert_tonal"].set(False)
                        self.apply_style()
                        break
            
            preset_combo.bind("<<ComboboxSelected>>", apply_preset)
            
            # Update scrollregion after all controls are added
            scrollable_frame.update_idletasks()
            canvas.configure(scrollregion=canvas.bbox("all"))

        elif "Style 5" in style:  # ASCII Art
            self.param_controls["char_density"] = self.create_slider("Character Density", 1, 3, 1)
            self.param_controls["char_size"] = self.create_slider("Character Size", 6, 16, 10)
            self.param_controls["contrast"] = self.create_slider("Contrast", 50, 200, 120)
            self.param_controls["brightness"] = self.create_slider("Brightness", 50, 150, 100)
            
            # Option to use extended ASCII
            options_frame = ttk.Frame(self.param_frame)
            options_frame.pack(fill=tk.X, padx=5, pady=5)
            
            self.param_controls["invert"] = tk.BooleanVar(value=False)
            ttk.Checkbutton(
                options_frame,
                text="Invert (Light to Dark)",
                variable=self.param_controls["invert"],
                command=self.apply_style
            ).pack(anchor=tk.W, padx=5, pady=2)
            
            # Add presets for different ASCII looks
            presets_frame = ttk.LabelFrame(self.param_frame, text="Presets")
            presets_frame.pack(fill=tk.X, padx=5, pady=5)
            
            preset_values = [
                ("Standard", 2, 10, 120, 100, False),
                ("Low Density", 3, 12, 130, 100, False),
                ("Bold Characters", 3, 16, 140, 90, False),
                ("High Contrast", 2, 10, 180, 100, False),
                ("Inverted", 2, 10, 120, 100, True)
            ]
            
            self.ascii_preset_var = tk.StringVar(value="Custom")
            preset_combo = ttk.Combobox(
                presets_frame,
                textvariable=self.ascii_preset_var,
                values=[preset[0] for preset in preset_values],
                state="readonly"
            )
            preset_combo.pack(fill=tk.X, pady=5)
            
            # Function to apply preset
            def apply_ascii_preset(event):
                preset_name = self.ascii_preset_var.get()
                for preset in preset_values:
                    if preset[0] == preset_name:
                        self.param_controls["char_density"].set(preset[1])
                        self.param_controls["contrast"].set(preset[2])
                        self.param_controls["brightness"].set(preset[3])
                        self.param_controls["char_set"].set(preset[4])
                        self.param_controls["use_extended"].set(preset[5])
                        self.param_controls["invert"].set(preset[6])
                        self.apply_style()
                        break
            
            preset_combo.bind("<<ComboboxSelected>>", apply_ascii_preset)

    def _update_mosaic_parameters(self, init_mode=False):
        """Update the mosaic parameters based on the selected mosaic type."""
        # Get the current mosaic type
        mosaic_type = self.mosaic_type_var.get()
        
        # Remove existing mosaic-specific parameters except common ones
        to_remove = []
        
        for key, control in self.param_controls.items():
            if key not in ["contrast", "brightness", "saturation", "units"]:
                # Keep track of keys to remove
                to_remove.append(key)
        
        # Remove the controls
        for key in to_remove:
            if key in self.param_controls:
                del self.param_controls[key]
        
        # Pattern type is common to both mosaic types
        # If a pattern_frame already exists, don't create a new one
        pattern_frame_exists = hasattr(self, 'pattern_frame') and self.pattern_frame.winfo_exists()
        
        if not pattern_frame_exists:
            self.pattern_frame = ttk.Frame(self.param_frame)
            self.pattern_frame.pack(fill=tk.X, padx=5, pady=5)
            ttk.Label(self.pattern_frame, text="Pattern:").pack(anchor=tk.W)
            
            # Get or create pattern type variable
            try:
                pattern_type = self.pattern_type_var.get()
            except (AttributeError, tk.TclError):
                pattern_type = "grid"
                
            self.pattern_type_var = tk.StringVar(value=pattern_type)
            pattern_combo = ttk.Combobox(
                self.pattern_frame,
                textvariable=self.pattern_type_var,
                values=["vertical", "horizontal", "grid"],
                state="readonly"
            )
            pattern_combo.pack(fill=tk.X, pady=5)
            pattern_combo.bind("<<ComboboxSelected>>", lambda e: self.apply_style())
        
        # Add type-specific parameters
        if mosaic_type == "slice":
            self.param_controls["edge_intensity"] = self.create_slider("Edge Intensity", 0, 200, 100)
            self.param_controls["x_shift"] = self.create_slider("X Shift", -100, 100, 0)
            self.param_controls["y_shift"] = self.create_slider("Y Shift", -100, 100, 30)
            self.param_controls["scale_shift"] = self.create_slider("Scale Shift", 0, 100, 0)
        
        # Apply the style if not in initialization mode
        if not init_mode:
            self.apply_style()

    # Mask Drawing Functions
    def toggle_mask_mode(self):
        """Toggle mask drawing mode on/off."""
        self.mask_mode = self.mask_var.get()
        
        if self.mask_mode:
            self.mask_radius = self.mask_radius_var.get()
            self.mask_feather = self.mask_feather_var.get()
            tool_name = "brush" if self.mask_tool_mode == "brush" else "lasso"
            self.status_var.set(f"Mask mode active with {tool_name} tool. Draw to protect areas from blur effect.")
            # Change cursor to indicate drawing mode
            self.canvas.config(cursor="pencil" if self.mask_tool_mode == "brush" else "crosshair")
        else:
            self.status_var.set("Mask mode disabled.")
            # Reset cursor
            self.canvas.config(cursor="")
            # Apply the style to update the image
            self.apply_style()

    def start_draw(self, event):
        """Start drawing a mask."""
        if not self.mask_mode or not self.original_image:
            return
        
        self.drawing = True
        
        # Handle brush mode
        if self.mask_tool_mode == "brush":
            # Create a new mask if it doesn't exist yet
            if not self.masks:
                # Create new mask for the current image size
                width, height = self.original_image.size
                new_mask = Image.new('L', (width, height), 0)
                self.masks.append(new_mask)
            
            # Draw at the initial position
            self.draw(event)
        
        # Handle lasso mode
        elif self.mask_tool_mode == "lasso":
            # Clear any previous lasso points
            self.lasso_points = []
            
            # Convert canvas coordinates to image coordinates
            img_coords = self.canvas_to_image_coords(event.x, event.y)
            if img_coords:
                # Add the first point
                self.lasso_points.append(img_coords)
                
                # Draw the first point on canvas
                canvas_width = self.canvas.winfo_width()
                canvas_height = self.canvas.winfo_height()
                self.temp_lasso_line = self.canvas.create_line(
                    event.x, event.y, event.x, event.y,
                    fill="white", width=1, dash=(4, 4)
                )

    def draw(self, event):
        """Continue drawing a mask."""
        if not self.drawing or not self.mask_mode or not self.original_image:
            return
        
        # Handle brush mode
        if self.mask_tool_mode == "brush":
            # Convert canvas coordinates to image coordinates
            img_coords = self.canvas_to_image_coords(event.x, event.y)
            if not img_coords:
                return
                
            img_x, img_y = img_coords
            
            # Update the current mask
            current_mask = self.masks[-1]
            # Import ImageDraw directly in the method to ensure it's available
            from PIL import ImageDraw
            draw = ImageDraw.Draw(current_mask)
            
            # Get current brush size and feather
            brush_radius = self.mask_radius_var.get()
            feather = self.mask_feather_var.get()
            
            # Draw a filled circle on the mask
            left = img_x - brush_radius
            top = img_y - brush_radius
            right = img_x + brush_radius
            bottom = img_y + brush_radius
            
            draw.ellipse([left, top, right, bottom], fill=255)
            
            # Apply the feathering by blurring the mask
            if feather > 0:
                self.masks[-1] = current_mask.filter(ImageFilter.GaussianBlur(radius=feather/2))
            
            # Update the composite mask
            self.update_composite_mask()
            
            # Apply the style to see the effect immediately - without visualization
            self.apply_style()
        
        # Handle lasso mode
        elif self.mask_tool_mode == "lasso":
            # Convert canvas coordinates to image coordinates
            img_coords = self.canvas_to_image_coords(event.x, event.y)
            if not img_coords:
                return
                
            # Add the point to the lasso points list
            self.lasso_points.append(img_coords)
            
            # Update the lasso line visualization
            if len(self.lasso_points) > 1:
                # Get canvas coordinates for the last two points
                prev_img_x, prev_img_y = self.lasso_points[-2]
                img_x, img_y = self.lasso_points[-1]
                
                # Convert back to canvas coordinates
                prev_canvas_x, prev_canvas_y = self.image_to_canvas_coords(prev_img_x, prev_img_y)
                canvas_x, canvas_y = self.image_to_canvas_coords(img_x, img_y)
                
                # Delete previous line
                if self.temp_lasso_line:
                    self.canvas.delete(self.temp_lasso_line)
                
                # Draw lines connecting all points
                points = []
                for point in self.lasso_points:
                    cx, cy = self.image_to_canvas_coords(point[0], point[1])
                    points.extend([cx, cy])
                
                self.temp_lasso_line = self.canvas.create_line(
                    points, fill="white", width=1, dash=(4, 4)
                )

    
    def stop_draw(self, event):
        """Stop drawing a mask."""
        if not self.drawing or not self.mask_mode:
            return
            
        self.drawing = False
        
        # Handle brush mode
        if self.mask_tool_mode == "brush":
            # Apply the style to update the image with the new mask
            self.apply_style()
        
        # Handle lasso mode
        elif self.mask_tool_mode == "lasso":
            # Need at least 3 points to create a polygon
            if len(self.lasso_points) < 3:
                # Clear the lasso operation
                if self.temp_lasso_line:
                    self.canvas.delete(self.temp_lasso_line)
                    self.temp_lasso_line = None
                self.lasso_points = []
                return
            
            # Close the lasso by connecting to the first point
            self.lasso_points.append(self.lasso_points[0])
            
            # Create or get a mask
            if not self.masks:
                # Create new mask for the current image size
                width, height = self.original_image.size
                new_mask = Image.new('L', (width, height), 0)
                self.masks.append(new_mask)
            else:
                new_mask = self.masks[-1]
            
            # Draw the polygon on the mask
            from PIL import ImageDraw
            draw = ImageDraw.Draw(new_mask)
            draw.polygon(self.lasso_points, fill=255)
            
            # Apply feathering if needed
            feather = self.mask_feather_var.get()
            if feather > 0:
                self.masks[-1] = new_mask.filter(ImageFilter.GaussianBlur(radius=feather/2))
            
            # Update the composite mask
            self.update_composite_mask()
            
            # Clear the lasso visualization
            if self.temp_lasso_line:
                self.canvas.delete(self.temp_lasso_line)
                self.temp_lasso_line = None
            
            # Reset lasso points
            self.lasso_points = []
            
            # Apply the style to update the image
            self.apply_style()

    def clear_masks(self):
        """Clear all masks."""
        self.masks = []
        self.mask_composite = None
        self.status_var.set("All masks cleared.")
        
        # Clear any lasso points
        self.lasso_points = []
        if hasattr(self, 'temp_lasso_line') and self.temp_lasso_line:
            self.canvas.delete(self.temp_lasso_line)
            self.temp_lasso_line = None
        
        # Apply the style without masks
        self.apply_style()

    def update_composite_mask(self):
        """Update the composite mask from all individual masks."""
        if not self.masks:
            self.mask_composite = None
            return
        
        # Start with a blank composite
        width, height = self.original_image.size
        self.mask_composite = Image.new('L', (width, height), 0)
        
        # Combine all masks (taking the maximum value at each pixel)
        for mask in self.masks:
            # Convert masks to numpy arrays for faster processing
            mask_array = np.array(mask)
            composite_array = np.array(self.mask_composite)
            
            # Take the maximum value at each pixel
            combined_array = np.maximum(mask_array, composite_array)
            
            # Convert back to PIL
            self.mask_composite = Image.fromarray(combined_array)

    def create_slider(self, label, min_val, max_val, default_val):
        frame = ttk.Frame(self.param_frame)
        frame.pack(fill=tk.X, padx=5, pady=5)

        ttk.Label(frame, text=label).pack(anchor=tk.W)

        var = tk.IntVar(value=default_val)
        slider = Scale(
            frame,
            from_=min_val,
            to=max_val,
            orient=tk.HORIZONTAL,
            variable=var,
            command=lambda _: self.apply_style()
        )
        slider.pack(fill=tk.X)

        return var
    
    def create_checkbox(self, label, default_val):
        var = tk.BooleanVar(value=default_val)
        checkbox = ttk.Checkbutton(
            self.param_frame, 
            text=label, 
            variable=var,
            command=self.apply_style
        )
        checkbox.pack(anchor=tk.W, padx=5, pady=5)
        
        return var
        
    def open_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.bmp *.gif"),
                ("All files", "*.*")
            ]
        )
        
        if file_path:
            try:
                self.original_image = Image.open(file_path)
                # Convert to RGB mode to ensure compatibility
                if self.original_image.mode != 'RGB':
                    self.original_image = self.original_image.convert('RGB')
                self.display_image(self.original_image)
                self.status_var.set(f"Image loaded: {file_path}")
                self.reset_image()
                
                # Clear masks when loading a new image
                self.masks = []
                self.mask_composite = None
                
                # Clear any lasso points
                self.lasso_points = []
                if hasattr(self, 'temp_lasso_line') and self.temp_lasso_line:
                    self.canvas.delete(self.temp_lasso_line)
                    self.temp_lasso_line = None
                    
            except Exception as e:
                self.status_var.set(f"Error loading image: {str(e)}")
                import traceback
                traceback.print_exc()


    def save_image(self):
        if not self.processed_image:
            return

        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[
                ("PNG files", "*.png"),
                ("JPEG files", "*.jpg *.jpeg"),
                ("All files", "*.*")
            ]
        )

        if file_path:
            self.processed_image.save(file_path)
            self.status_var.set(f"Image saved to: {file_path}")
    
    def display_image(self, image):
        try:
            # Resize image to fit canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width <= 1 or canvas_height <= 1:
                # Canvas hasn't been drawn yet, use default size
                canvas_width = 800
                canvas_height = 600
            
            # Calculate resize ratio
            img_width, img_height = image.size
            ratio = min(canvas_width/img_width, canvas_height/img_height)
            new_width = int(img_width * ratio)
            new_height = int(img_height * ratio)
            
            # Ensure we're working with RGB mode for display
            if image.mode != 'RGB':
                display_image = image.convert('RGB')
            else:
                display_image = image.copy()
                
            resized_image = display_image.resize((new_width, new_height), Image.LANCZOS)
            self.displayed_image = ImageTk.PhotoImage(resized_image)
            
            # Update canvas
            self.canvas.delete("all")
            self.canvas.create_image(
                canvas_width // 2, 
                canvas_height // 2, 
                anchor=tk.CENTER, 
                image=self.displayed_image
            )
            
            # Update status
            if hasattr(image, 'filename'):
                self.status_var.set(f"Image: {image.filename} | Size: {img_width}x{img_height}")
        except Exception as e:
            self.status_var.set(f"Error displaying image: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def reset_image(self):
        if self.original_image:
            self.processed_image = self.original_image.copy()
            self.display_image(self.original_image)
            self.style_var.set("None")
            self.initialize_parameters()
            
            # Clear masks when resetting
            self.masks = []
            self.mask_composite = None
            
            # Clear any lasso points
            self.lasso_points = []
            if hasattr(self, 'temp_lasso_line') and self.temp_lasso_line:
                self.canvas.delete(self.temp_lasso_line)
                self.temp_lasso_line = None
    
    def canvas_to_image_coords(self, canvas_x, canvas_y):
        """Convert canvas coordinates to image coordinates."""
        if not self.original_image:
            return None
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # Get image dimensions
        img_width, img_height = self.original_image.size
        
        # Calculate scale and position
        scale = min(canvas_width/img_width, canvas_height/img_height)
        
        # Calculate where the scaled image is positioned on the canvas
        scaled_width = int(img_width * scale)
        scaled_height = int(img_height * scale)
        
        # Calculate offset from canvas center
        canvas_center_x = canvas_width // 2
        canvas_center_y = canvas_height // 2
        image_x0 = canvas_center_x - (scaled_width // 2)
        image_y0 = canvas_center_y - (scaled_height // 2)
        
        # Convert canvas coordinates to image coordinates
        img_x = int((canvas_x - image_x0) / scale)
        img_y = int((canvas_y - image_y0) / scale)
        
        # Ensure coordinates are within the image bounds
        if 0 <= img_x < img_width and 0 <= img_y < img_height:
            return (img_x, img_y)
        return None

    def image_to_canvas_coords(self, img_x, img_y):
        """Convert image coordinates to canvas coordinates."""
        if not self.original_image:
            return (0, 0)
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # Get image dimensions
        img_width, img_height = self.original_image.size
        
        # Calculate scale and position
        scale = min(canvas_width/img_width, canvas_height/img_height)
        
        # Calculate where the scaled image is positioned on the canvas
        scaled_width = int(img_width * scale)
        scaled_height = int(img_height * scale)
        
        # Calculate offset from canvas center
        canvas_center_x = canvas_width // 2
        canvas_center_y = canvas_height // 2
        image_x0 = canvas_center_x - (scaled_width // 2)
        image_y0 = canvas_center_y - (scaled_height // 2)
        
        # Convert image coordinates to canvas coordinates
        canvas_x = int(img_x * scale + image_x0)
        canvas_y = int(img_y * scale + image_y0)
        
        return (canvas_x, canvas_y)

    def apply_style(self, update_mask_only=False):
        if not self.original_image:
            return
        
        style = self.style_var.get()
        self.current_style = style
        
        # If style is changing and we're not just updating the mask, update parameters
        if "Style" in style and (not hasattr(self, 'last_style') or self.last_style != style):
            self.initialize_parameters()
            self.last_style = style

            # Update status with style description
            if style in self.style_descriptions:
                self.status_var.set(self.style_descriptions[style])
        
        # Start with original image
        try:
            img = self.original_image.copy()
            # Ensure we're working with an RGB image
            if img.mode != 'RGB':
                img = img.convert('RGB')
        except Exception as e:
            self.status_var.set(f"Error processing image: {str(e)}")
            return
        
        # Get common parameter values
        contrast = self.param_controls.get("contrast", tk.IntVar(value=100)).get() / 100
        brightness = self.param_controls.get("brightness", tk.IntVar(value=100)).get() / 100
        saturation = self.param_controls.get("saturation", tk.IntVar(value=100)).get() / 100
        
        # Apply common adjustments
        img = ImageEnhance.Contrast(img).enhance(contrast)
        img = ImageEnhance.Brightness(img).enhance(brightness)
        img = ImageEnhance.Color(img).enhance(saturation)
        
        # Apply style-specific transformations
        if "Style 1" in style:  # Abstract Portrait / Datamosh
            img = self.apply_abstract_portrait_style(img, contrast)
        elif "Style 2" in style:  # Colorful Blur
            img = self.apply_colorful_blur_style(img)
        elif "Style 3" in style:  # Mosaic Effect
            img = self.apply_mosaic_effect_style(img)
        elif "Style 4" in style:  # Sabattier Effect
            img = self.apply_sabattier_effect_style(img)
        elif "Style 5" in style:  # ASCII Art
            img = self.apply_ascii_art_style(img)

        # Store the processed image
        self.processed_image = img
        
        # Always display the processed image without any overlay
        self.display_image(img)

    def apply_abstract_portrait_style(self, img, contrast):
        """Apply the Abstract Portrait style to the image with improved glitch aesthetics.
        Creates a more dynamic, varied block structure with intentional artifacts
        that better match the reference images. Removes white line artifacts and edge preservation."""
        # Get parameters
        pixelate = self.param_controls.get("pixelate", tk.IntVar(value=5)).get()
        block_variety = self.param_controls.get("block_variety", tk.IntVar(value=50)).get() / 100
        glitch_amount = self.param_controls.get("glitch_amount", tk.IntVar(value=70)).get() / 100
        blur_amount = self.param_controls.get("blur", tk.IntVar(value=0)).get() / 100
        motion_blur = self.param_controls.get("motion_blur", tk.IntVar(value=0)).get() / 100
        artifact_amount = self.param_controls.get("artifact_amount", tk.IntVar(value=30)).get() / 100
        random_seed = self.param_controls.get("random_seed", tk.IntVar(value=42)).get()
        
        # Set random seed for reproducible results
        np.random.seed(random_seed)
        
        # Convert to numpy array for processing
        img_array = np.array(img)
        
        # Always convert to grayscale first
        gray_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        
        # Apply blur to base image if requested
        if blur_amount > 0:
            blur_radius = int(1 + blur_amount * 20)
            if blur_radius % 2 == 0:
                blur_radius += 1  # Ensure odd kernel size for GaussianBlur
            gray_array = cv2.GaussianBlur(gray_array, (blur_radius, blur_radius), 0)
        
        # Apply motion blur if requested
        if motion_blur > 0:
            # Create motion blur kernel
            size = int(3 + motion_blur * 30)
            if size % 2 == 0:
                size += 1  # Ensure odd kernel size
            
            # Random angle for motion blur
            angle = np.random.randint(0, 180)
            
            # Create motion blur kernel
            kernel = np.zeros((size, size))
            center = size // 2
            
            # Create line in the kernel for motion blur effect
            for i in range(size):
                # Calculate x, y coordinates for the line
                x = int(center + (i - center) * np.cos(np.radians(angle)))
                y = int(center + (i - center) * np.sin(np.radians(angle)))
                
                # Ensure coordinates are within bounds
                if 0 <= x < size and 0 <= y < size:
                    kernel[y, x] = 1
            
            # Normalize kernel
            kernel = kernel / np.sum(kernel)
            
            # Apply motion blur
            gray_array = cv2.filter2D(gray_array, -1, kernel)
        
        # Create a true datamosh effect similar to P-frame/I-frame corruption
        h, w = gray_array.shape
        
        # STEP 1: Create patchwork/pixelation effect with more varied blocks
        # Use different approach with non-uniform grid and irregular blocks
        patchwork = np.zeros_like(gray_array)
        
        # Define wider range of block sizes based on variety parameter
        min_block = max(1, int(pixelate * (1 - block_variety * 0.8)))
        max_block = max(min_block + 2, int(pixelate * (1 + block_variety * 5)))  # More extreme variety
        
        # Create adaptive grid with varying block sizes
        y = 0
        while y < h:
            x = 0
            # Use different block heights for each row
            row_height = np.random.randint(min_block, max_block + 1)
            row_height = min(row_height, h - y)  # Ensure we don't go out of bounds
            
            while x < w:
                # Randomize block width for each cell
                block_width = np.random.randint(min_block, max_block + 1)
                block_width = min(block_width, w - x)  # Ensure we don't go out of bounds
                
                # Get average value for this block
                block_avg = np.mean(gray_array[y:y+row_height, x:x+block_width])
                
                # Apply the average value to the block
                patchwork[y:y+row_height, x:x+block_width] = block_avg
                
                # Move to next block
                x += block_width
            
            # Move to next row
            y += row_height
        
        # Apply custom enhanced effect for more varied distortion
        displacement_map = patchwork.copy()
        
        # STEP 2: Add intentional glitch artifacts (avoiding long white lines)
        num_artifacts = int(10 * artifact_amount)
        for _ in range(num_artifacts):
            if np.random.random() < 0.6:  # Vertical artifacts
                # Create a vertical artifact
                x = np.random.randint(0, w)
                width = np.random.randint(1, max(2, int(max_block * 0.5)))
                height = np.random.randint(h // 12, h // 3)  # Shorter height
                y_start = np.random.randint(0, h - height)
                
                # Choose what type of artifact to create
                artifact_type = np.random.random()
                
                if artifact_type < 0.3:  # Black artifact
                    displacement_map[y_start:y_start+height, x:x+width] = 0
                elif artifact_type < 0.4:  # Rarely create white artifacts (reduced probability)
                    # Make white artifacts much shorter
                    short_height = min(height, h // 20)
                    displacement_map[y_start:y_start+short_height, x:x+width] = 255
                else:  # Shifted artifact (copy from elsewhere)
                    src_x = np.random.randint(0, w - width)
                    src_y = np.random.randint(0, h - height)
                    displacement_map[y_start:y_start+height, x:x+width] = patchwork[src_y:src_y+height, src_x:src_x+width]
            
            else:  # Horizontal artifacts
                # Create a horizontal artifact
                y = np.random.randint(0, h)
                height = np.random.randint(1, max(2, int(max_block * 0.5)))
                width = np.random.randint(w // 12, w // 3)  # Shorter width
                x_start = np.random.randint(0, w - width)
                
                # Choose what type of artifact to create
                artifact_type = np.random.random()
                
                if artifact_type < 0.3:  # Black artifact
                    displacement_map[y:y+height, x_start:x_start+width] = 0
                elif artifact_type < 0.4:  # Rarely create white artifacts (reduced probability)
                    # Make white artifacts much shorter
                    short_width = min(width, w // 20)
                    displacement_map[y:y+height, x_start:x_start+short_width] = 255
                else:  # Shifted artifact (copy from elsewhere)
                    src_x = np.random.randint(0, w - width)
                    src_y = np.random.randint(0, h - height)
                    displacement_map[y:y+height, x_start:x_start+width] = patchwork[src_y:src_y+height, src_x:src_x+width]
        
        # STEP 3: Create more dramatic block shifts (like in reference images)
        num_shifts = int(30 * glitch_amount)
        for _ in range(num_shifts):
            # Select random rectangular region - vary sizes more dramatically
            rect_w = np.random.randint(w // 10, w // 2)  # Larger possible regions
            rect_h = np.random.randint(h // 10, h // 2)
            
            x = np.random.randint(0, w - rect_w)
            y = np.random.randint(0, h - rect_h)
            
            # More dramatic shift amounts
            shift_x = np.random.randint(-int(50 * glitch_amount), int(50 * glitch_amount))
            shift_y = np.random.randint(-int(30 * glitch_amount), int(30 * glitch_amount))
            
            # Create a shifted version
            x_dest = max(0, min(w - rect_w, x + shift_x))
            y_dest = max(0, min(h - rect_h, y + shift_y))
            
            # Copy the block to its new location
            displacement_map[y_dest:y_dest+rect_h, x_dest:x_dest+rect_w] = patchwork[y:y+rect_h, x:x+rect_w]
        
        # STEP 4: Apply displacement mapping with more extreme values for dramatic effect
        # Create X and Y displacement fields based on our displacement map
        scale_x = int(-130 * glitch_amount)  # More extreme displacement
        scale_y = int(80 * glitch_amount)
        
        # Normalize displacement map to [-1, 1] range
        disp_x = (displacement_map.astype(np.float32) / 127.5) - 1.0
        disp_y = (displacement_map.astype(np.float32) / 127.5) - 1.0
        
        # Apply scaling factors
        disp_x *= scale_x
        disp_y *= scale_y
        
        # Create the remapping coordinates
        map_x = np.zeros((h, w), dtype=np.float32)
        map_y = np.zeros((h, w), dtype=np.float32)
        
        for y in range(h):
            for x in range(w):
                map_x[y, x] = x + disp_x[y, x]
                map_y[y, x] = y + disp_y[y, x]
        
        # Apply the displacement mapping
        displaced = cv2.remap(gray_array, map_x, map_y, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
        
        # STEP 5: Apply a second displacement map with different parameters
        scale_x2 = int(110 * glitch_amount)
        scale_y2 = int(-150 * glitch_amount)
        
        # Create a second displacement map with different transformations
        displacement_map2 = np.zeros_like(displacement_map)
        
        # Apply vertical stretching/compression to create scan-line like effects
        for y in range(0, h, max(1, int(min_block * 0.7))):
            stretch_factor = 1.0 + 0.5 * artifact_amount * (np.random.random() - 0.5)
            
            # Calculate how many rows to affect
            rows = min(int(min_block * 2 * stretch_factor), h - y)
            
            if np.random.random() < 0.3 * artifact_amount:  # Chance of row artifact
                # Apply horizontal shift to entire row
                shift = np.random.randint(-int(20 * artifact_amount), int(20 * artifact_amount))
                for r in range(rows):
                    if y + r < h:
                        displacement_map2[y + r] = np.roll(displacement_map[y + r], shift)
            else:
                # Just copy
                for r in range(rows):
                    if y + r < h:
                        displacement_map2[y + r] = displacement_map[y + r]
        
        # Apply second displacement
        disp_x2 = (displacement_map2.astype(np.float32) / 127.5) - 1.0
        disp_y2 = (displacement_map2.astype(np.float32) / 127.5) - 1.0
        
        disp_x2 *= scale_x2
        disp_y2 *= scale_y2
        
        map_x2 = np.zeros((h, w), dtype=np.float32)
        map_y2 = np.zeros((h, w), dtype=np.float32)
        
        for y in range(h):
            for x in range(w):
                map_x2[y, x] = x + disp_x2[y, x]
                map_y2[y, x] = y + disp_y2[y, x]
        
        displaced2 = cv2.remap(gray_array, map_x2, map_y2, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
        
        # STEP 6: Blend layers with more complex compositing
        # Blend using "multiply" operation (like Photoshop's multiply blend)
        layer1 = displaced.astype(np.float32) / 255.0
        layer2 = displaced2.astype(np.float32) / 255.0
        
        # Multiply blend
        multiplied = layer1 * layer2
        
        # Convert back to 0-255 range
        result = (multiplied * 255).astype(np.uint8)
        
        # STEP 7: Add data corruption artifacts (damaged scan lines, but avoid long white lines)
        if artifact_amount > 0:
            num_line_artifacts = int(10 * artifact_amount)
            for _ in range(num_line_artifacts):
                if np.random.random() < 0.7:  # Horizontal line artifact
                    y_pos = np.random.randint(0, h)
                    line_thickness = np.random.randint(1, 3)
                    for i in range(line_thickness):
                        if y_pos + i < h:
                            if np.random.random() < 0.6:  # Prefer black lines over white
                                # Mostly black lines, rarely white and only in short segments
                                if np.random.random() < 0.8:  # 80% chance of black line
                                    # Black line
                                    start_x = np.random.randint(0, w // 2)
                                    length = np.random.randint(w // 10, w // 3)  # Shorter segments
                                    end_x = min(start_x + length, w)
                                    result[y_pos + i, start_x:end_x] = 0
                                else:
                                    # Very short white segment
                                    start_x = np.random.randint(0, w - w // 10)
                                    length = np.random.randint(5, w // 10)  # Very short segment
                                    end_x = min(start_x + length, w)
                                    result[y_pos + i, start_x:end_x] = 255
                            else:
                                # Shifted line
                                shift = np.random.randint(-40, 40)
                                result[y_pos + i] = np.roll(result[y_pos + i], shift)
                else:  # Vertical line artifact
                    x_pos = np.random.randint(0, w)
                    line_thickness = np.random.randint(1, 3)
                    line_height = np.random.randint(h // 10, h // 3)  # Shorter height
                    y_start = np.random.randint(0, h - line_height)
                    for i in range(line_thickness):
                        if x_pos + i < w:
                            if np.random.random() < 0.6:  # Prefer black lines over white
                                # Mostly black lines, rarely white and only in short segments
                                if np.random.random() < 0.8:  # 80% chance of black line
                                    # Black line
                                    result[y_start:y_start+line_height, x_pos + i] = 0
                                else:
                                    # Very short white segment
                                    short_height = min(line_height // 3, 10)  # Limit the white segment height
                                    short_start = y_start + np.random.randint(0, max(1, line_height - short_height))
                                    result[short_start:short_start+short_height, x_pos + i] = 255
                            else:
                                # Shifted section
                                shift = np.random.randint(-10, 10)
                                for y in range(y_start, min(y_start + line_height, h)):
                                    if 0 <= y + shift < h:
                                        result[y, x_pos + i] = result[y + shift, x_pos + i]
        
        # Apply contrast enhancement for stark black and white look
        normalized = result.astype(np.float32) / 255.0
        
        # Get contrast value (0.5 to 4.0 range for even stronger contrast)
        contrast_val = max(0.5, contrast / 40.0)  # More aggressive contrast scaling
        
        # Apply contrast adjustment
        mean_val = 0.5  # Use fixed middle gray
        contrasted = (normalized - mean_val) * contrast_val + mean_val
        
        # Clip to 0-1 range
        contrasted = np.clip(contrasted, 0, 1)
        
        # Convert back to 0-255 range
        result = (contrasted * 255).astype(np.uint8)
        
        # Convert to 3-channel grayscale for PIL
        result_rgb = cv2.cvtColor(result, cv2.COLOR_GRAY2RGB)
        
        # Convert back to PIL
        return Image.fromarray(result_rgb)

    def change_mask_tool(self):
        """Change the current mask drawing tool."""
        self.mask_tool_mode = self.mask_tool_var.get()
        
        # Reset any active lasso operation
        self.lasso_points = []
        if self.temp_lasso_line:
            self.canvas.delete(self.temp_lasso_line)
            self.temp_lasso_line = None
        
        # Update cursor and status message
        if self.mask_mode:
            self.canvas.config(cursor="pencil" if self.mask_tool_mode == "brush" else "crosshair")
            tool_name = "brush" if self.mask_tool_mode == "brush" else "lasso"
            self.status_var.set(f"Mask mode active with {tool_name} tool. Draw to protect areas from blur effect.")

    def apply_colorful_blur_style(self, img):
        """Apply the Colorful Blur style to the image, with optional masking.
        Masked areas remain sharp (protected from blur)."""
        blur_radius = self.param_controls.get("blur_radius", tk.IntVar(value=15)).get()
        color_shift = self.param_controls.get("color_shift", tk.IntVar(value=50)).get() / 100
        
        # Split into RGB channels
        r, g, b = img.split()
        
        # Create blurred version for each channel with different radii
        r_blur = r.filter(ImageFilter.GaussianBlur(radius=blur_radius * (1 + color_shift * 0.5)))
        g_blur = g.filter(ImageFilter.GaussianBlur(radius=blur_radius))
        b_blur = b.filter(ImageFilter.GaussianBlur(radius=blur_radius * (1 - color_shift * 0.5)))
        
        # Calculate offset for channel shifting
        img_width, img_height = img.size
        offset_x = int(img_width * 0.02 * color_shift)
        offset_y = int(img_height * 0.02 * color_shift)
        
        # Apply the channel offset
        r_offset = ImageChops.offset(r_blur, offset_x, offset_y)
        b_offset = ImageChops.offset(b_blur, -offset_x, -offset_y)
        
        # Create the fully blurred image by merging channels
        blurred_img = Image.merge("RGB", (r_offset, g_blur, b_offset))
        # Enhance saturation of the blurred image
        blurred_img = ImageEnhance.Color(blurred_img).enhance(1.5)
        
        # If we have masks, apply them (inverted - masks protect from blur)
        if self.mask_composite:
            # Invert the mask (so that drawn areas are protected from blur)
            inverted_mask = ImageOps.invert(self.mask_composite)
            
            # Composite the original and blurred images using the inverted mask
            # Areas with mask value 255 (white) will show the original image
            # Areas with mask value 0 (black) will show the blurred image
            result = Image.composite(img, blurred_img, self.mask_composite)
            return result
        else:
            # No mask, apply the effect to the whole image
            return blurred_img

    def apply_sabattier_effect_style(self, img):
        """Apply the Sabattier effect (photographic solarization) to the image.
        
        This simulates the darkroom technique where a partially developed photograph
        is briefly exposed to light before completing development, creating a 
        partial reversal of tones with distinctive edge effects.
        """
        # Get parameters with proper defaults to prevent errors
        threshold = self.param_controls.get("threshold", tk.IntVar(value=100)).get()
        edge_strength = self.param_controls.get("edge_strength", tk.IntVar(value=30)).get() / 100
        contrast = self.param_controls.get("contrast", tk.IntVar(value=130)).get() / 100
        brightness = self.param_controls.get("brightness", tk.IntVar(value=100)).get() / 100
        developer_variation = self.param_controls.get("developer_variation", tk.IntVar(value=0)).get() / 100
        grain_amount = self.param_controls.get("grain_amount", tk.IntVar(value=0)).get()
        
        # Get parameters for selective inversion
        invert_shadows = self.param_controls.get("invert_shadows", tk.BooleanVar(value=False)).get()
        invert_midtones = self.param_controls.get("invert_midtones", tk.BooleanVar(value=True)).get()
        invert_highlights = self.param_controls.get("invert_highlights", tk.BooleanVar(value=False)).get()
        shadow_threshold = self.param_controls.get("shadow_threshold", tk.IntVar(value=64)).get()
        highlight_threshold = self.param_controls.get("highlight_threshold", tk.IntVar(value=192)).get()
        
        # Get tonal range selection parameters
        tonal_min = self.param_controls.get("tonal_min", tk.IntVar(value=0)).get()
        tonal_max = self.param_controls.get("tonal_max", tk.IntVar(value=255)).get()
        invert_tonal = self.param_controls.get("invert_tonal", tk.BooleanVar(value=False)).get()
        
        # Convert to numpy array for processing
        img_array = np.array(img)
        
        # Convert to grayscale
        if len(img_array.shape) == 3:
            gray_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        else:
            gray_array = img_array.copy()
        
        # Store a copy of the original grayscale for later blending
        original_gray = gray_array.copy()
        
        # Create a high contrast version of the grayscale image
        mean = np.mean(gray_array)
        high_contrast = np.clip((gray_array.astype(np.float32) - mean) * 1.5 + mean, 0, 255).astype(np.uint8)
        
        # Create selective masks based on tonal range
        shadow_mask = gray_array < shadow_threshold
        midtone_mask = (gray_array >= shadow_threshold) & (gray_array <= highlight_threshold)
        highlight_mask = gray_array > highlight_threshold
        
        # Apply selective inversion based on user choices
        solarized = gray_array.copy()
        
        if invert_shadows:
            solarized[shadow_mask] = 255 - solarized[shadow_mask]
        
        if invert_midtones:
            solarized[midtone_mask] = 255 - solarized[midtone_mask]
        
        if invert_highlights:
            solarized[highlight_mask] = 255 - solarized[highlight_mask]
        
        # Detect edges for the distinctive solarization edge effect
        sobelx = cv2.Sobel(high_contrast, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(high_contrast, cv2.CV_64F, 0, 1, ksize=3)
        edge_magnitude = np.sqrt(sobelx**2 + sobely**2)
        
        # Normalize the edge magnitude
        edge_magnitude = cv2.normalize(edge_magnitude, None, 0, 255, cv2.NORM_MINMAX)
        
        # Create the edge effect by creating lighter edges (characteristic of Sabattier)
        edge_effect = 255 - (edge_magnitude * edge_strength)
        edge_effect = edge_effect.astype(np.uint8)
        
        # Screen blend mode to lighten edges (mimics the Sabattier light edges)
        # Screen blend formula: 255 - ((255 - A) * (255 - B) / 255)
        inv_solarized = 255 - solarized
        inv_edge = 255 - edge_effect
        screen_blend = 255 - ((inv_solarized.astype(np.float32) * inv_edge.astype(np.float32)) / 255)
        screen_blend = np.clip(screen_blend, 0, 255).astype(np.uint8)
        
        # Apply contrast adjustment
        mean_val = np.mean(screen_blend)
        contrasted = np.clip((screen_blend.astype(np.float32) - mean_val) * contrast + mean_val, 0, 255).astype(np.uint8)
        
        # Apply brightness adjustment
        brightened = np.clip(contrasted.astype(np.float32) * brightness, 0, 255).astype(np.uint8)
        
        # Create uneven developer effect (if enabled)
        if developer_variation > 0:
            # Generate smooth noise pattern
            height, width = brightened.shape
            noise_scale = max(1, min(width, height) // 20)
            base_noise = np.random.random((height // noise_scale + 1, width // noise_scale + 1))
            noise = cv2.resize(base_noise, (width, height), interpolation=cv2.INTER_LINEAR)
            noise = cv2.GaussianBlur(noise, (0, 0), sigmaX=noise_scale/4)
            
            # Normalize and scale
            noise = noise - np.mean(noise)
            noise = noise * developer_variation
            
            # Apply as subtle variation in contrast
            brightened = np.clip(brightened * (1.0 + noise), 0, 255).astype(np.uint8)
        
        # Add film grain if enabled
        if grain_amount > 0:
            noise = np.random.normal(0, grain_amount / 5, brightened.shape).astype(np.int16)
            brightened = np.clip(brightened.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        # Convert back to PIL image
        result = Image.fromarray(brightened)
        
        # Apply tonal range selection if requested
        if tonal_min > 0 or tonal_max < 255:
            # Create mask based on tonal range
            tonal_mask = (original_gray >= tonal_min) & (original_gray <= tonal_max)
            if invert_tonal:
                tonal_mask = ~tonal_mask
            
            # Convert to PIL mask
            mask_array = np.zeros_like(original_gray, dtype=np.uint8)
            mask_array[tonal_mask] = 255
            mask_img = Image.fromarray(mask_array)
            
            # Blend based on mask
            original_img = Image.fromarray(original_gray)
            result = Image.composite(result, original_img, mask_img)
        
        # Apply brush mask if present
        if hasattr(self, 'mask_composite') and self.mask_composite is not None:
            # Blend with original image based on mask
            original_img = Image.fromarray(original_gray)
            result = Image.composite(original_img, result, self.mask_composite)
        
        return result

    def apply_ascii_art_style(self, img):
        """Apply ASCII art style to the image using block characters.
        
        This transforms the image into a pattern of block characters (█, ▓, ▒, ░)
        against a white background, preserving the original image proportions.
        """
        from PIL import ImageFont
        import math
        
        # Get parameters
        char_density = self.param_controls.get("char_density", tk.IntVar(value=3)).get()
        char_size = self.param_controls.get("char_size", tk.IntVar(value=16)).get()
        brightness = self.param_controls.get("brightness", tk.IntVar(value=100)).get() / 100.0
        contrast = self.param_controls.get("contrast", tk.IntVar(value=120)).get() / 100.0
        invert = self.param_controls.get("invert", tk.BooleanVar(value=False)).get()
        
        # Convert to grayscale
        if img.mode != 'L':
            gray_img = img.convert('L')
        else:
            gray_img = img.copy()
        
        # Adjust contrast and brightness
        enhancer = ImageEnhance.Contrast(gray_img)
        gray_img = enhancer.enhance(contrast)
        enhancer = ImageEnhance.Brightness(gray_img)
        gray_img = enhancer.enhance(brightness)
        
        # Define block characters (from darkest to lightest)
        chars = ['█', '▓', '▒', '░', ' ']
        
        # Invert if requested
        if invert:
            chars = chars[::-1]
        
        # Calculate the width and height of the original image
        orig_width, orig_height = gray_img.size
        
        # Calculate base cell size based on density
        base_cell_size = 8 * char_density
        
        # Keep original aspect ratio by using the same cell size for width and height
        # Calculate dimensions based on a target number of characters
        target_total_chars = 3000 / char_density  # Fewer chars for lower density
        
        # Calculate what the width and height would be with uniform cell size
        initial_ascii_width = orig_width / base_cell_size
        initial_ascii_height = orig_height / base_cell_size
        
        # Scale to target character count while preserving aspect ratio
        scale_factor = math.sqrt(target_total_chars / (initial_ascii_width * initial_ascii_height))
        
        # Final dimensions
        ascii_width = max(10, int(initial_ascii_width * scale_factor))
        ascii_height = max(10, int(initial_ascii_height * scale_factor))
        
        # Create a new image for the ASCII result with white background
        # Calculate font size based on user preference
        font_size = char_size * 2
        
        # Output image dimensions - keep the aspect ratio correct
        font_aspect_ratio = 0.6  # Typical monospace font width/height ratio
        output_width = int(ascii_width * font_size * font_aspect_ratio)
        output_height = int(ascii_height * font_size)
        
        # Create a new image with white background
        ascii_img = Image.new('RGB', (output_width, output_height), color='white')
        draw = ImageDraw.Draw(ascii_img)
        
        try:
            # Try to load a monospace font
            try:
                # Try common monospace fonts
                font_paths = [
                    "/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf",
                    "/Library/Fonts/Courier New.ttf",
                    "C:\\Windows\\Fonts\\cour.ttf",
                    "/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf",
                ]
                
                font = None
                for path in font_paths:
                    try:
                        font = ImageFont.truetype(path, font_size)
                        break
                    except (IOError, OSError):
                        continue
                        
                if font is None:
                    # Fall back to default if no font found
                    font = ImageFont.load_default()
            except Exception:
                # Fall back to default if any error occurs
                font = ImageFont.load_default()
            
            # Resize the image before processing for better performance
            # Use LANCZOS for high-quality downsampling
            small_img = gray_img.resize((ascii_width, ascii_height), Image.LANCZOS)
            img_array = np.array(small_img)
            
            # Process the image and convert to ASCII
            for y in range(ascii_height):
                for x in range(ascii_width):
                    # Get pixel brightness directly from resized image
                    if y < img_array.shape[0] and x < img_array.shape[1]:
                        brightness_val = img_array[y, x]
                        
                        # Map brightness to a character
                        # Map to 0-4 range (for 5 characters)
                        char_index = int(brightness_val / 255 * 4)
                        char_index = max(0, min(char_index, 4))  # Ensure within bounds
                        char = chars[char_index]
                        
                        # Only draw non-space characters to improve performance
                        if char != ' ':
                            # Draw the character with black color (on white background)
                            draw_x = int(x * font_size * font_aspect_ratio)
                            draw_y = y * font_size
                            draw.text((draw_x, draw_y), char, fill='black', font=font)
            
        except Exception as e:
            # Fallback if there are any errors
            print(f"Error in ASCII conversion: {str(e)}")
            # Return original image with text explaining the error
            error_img = Image.new('RGB', img.size, color='white')
            error_draw = ImageDraw.Draw(error_img)
            error_draw.text((10, 10), f"ASCII conversion error: {str(e)}", fill='black')
            return error_img
        
        # Return the ASCII art image
        return ascii_img

    def apply_mosaic_effect_style(self, img):
        # Get common parameters
        units = self.param_controls.get("units", tk.IntVar(value=20)).get()
        use_feathering = getattr(self, "feather_var", tk.BooleanVar(value=False)).get()
        pattern_type = getattr(self, "pattern_type_var", tk.StringVar(value="grid")).get()
        
        # Get type-specific parameters
        mosaic_type = getattr(self, "mosaic_type_var", tk.StringVar(value="flip")).get()
        
        if mosaic_type == "flip":
            # Flip-based mosaic
            img = self.create_flip_mosaic(img.copy(), units, pattern_type, use_feathering)
                
        else:  # slice
            # Slice-based mosaic
            edge_intensity = self.param_controls.get("edge_intensity", tk.IntVar(value=100)).get() / 100
            x_shift = self.param_controls.get("x_shift", tk.IntVar(value=0)).get() / 100
            y_shift = self.param_controls.get("y_shift", tk.IntVar(value=30)).get() / 100
            scale_shift = self.param_controls.get("scale_shift", tk.IntVar(value=0)).get() / 100
            
            # Create slice distortion effect
            img = self.create_slice_mosaic(
                img, units, x_shift, y_shift, scale_shift, 
                edge_intensity, use_feathering, pattern_type
            )
        
        return img

    def create_flip_mosaic(self, image, units, pattern_type, use_feathering):
        """
        Create a flip-based mosaic effect.
        """
        width, height = image.size
        result = image.copy()
        
        if pattern_type == 'vertical':
            strip_width = width // units
            
            for i in range(units):
                left = i * strip_width
                right = (i + 1) * strip_width if i < units - 1 else width
                
                # Crop the strip
                strip = image.crop((left, 0, right, height))
                
                # Mirror the strip horizontally
                mirrored_strip = ImageOps.mirror(strip)
                
                # Create feathered edges if requested
                if use_feathering:
                    # Create a mask for feathered edges (only at the boundaries)
                    mask = Image.new('L', (right - left, height), 255)
                    mask_array = np.array(mask)
                    
                    # Calculate feathering width (in pixels)
                    feather_width = max(1, int(strip_width * 0.1))  # Fixed at 10% of strip width
                    
                    # Only feather at the strip boundaries with adjacent strips
                    if i > 0:  # Left edge (only if not first strip)
                        for x in range(feather_width):
                            # Linear gradient from 0 to 255
                            alpha = int(255 * x / feather_width)
                            mask_array[:, x] = alpha
                    
                    if i < units - 1:  # Right edge (only if not last strip)
                        for x in range(feather_width):
                            # Linear gradient from 255 to 0
                            alpha = int(255 * (feather_width - x) / feather_width)
                            if right - left - x - 1 >= 0:  # Ensure we're in bounds
                                mask_array[:, right - left - x - 1] = alpha
                    
                    # Convert back to PIL mask
                    mask = Image.fromarray(mask_array)
                    
                    # Apply mask to the mirrored strip
                    result.paste(mirrored_strip, (left, 0), mask)
                else:
                    # No feathering, simply paste the mirrored strip
                    result.paste(mirrored_strip, (left, 0))
        
        elif pattern_type == 'horizontal':
            strip_height = height // units
            
            for i in range(units):
                top = i * strip_height
                bottom = (i + 1) * strip_height if i < units - 1 else height
                
                # Crop the strip
                strip = image.crop((0, top, width, bottom))
                
                # Flip the strip vertically
                flipped_strip = ImageOps.flip(strip)
                
                # Create feathered edges if requested
                if use_feathering:
                    # Create a gradient mask for feathered edges
                    mask = Image.new('L', (width, bottom - top), 255)
                    mask_array = np.array(mask)
                    
                    # Calculate feathering height (in pixels)
                    feather_height = max(1, int(strip_height * 0.1))  # Fixed at 10% of strip height
                    
                    # Only feather at the strip boundaries with adjacent strips
                    if i > 0:  # Top edge (only if not first strip)
                        for y in range(feather_height):
                            # Linear gradient from 0 to 255
                            alpha = int(255 * y / feather_height)
                            mask_array[y, :] = alpha
                    
                    if i < units - 1:  # Bottom edge (only if not last strip)
                        for y in range(feather_height):
                            # Linear gradient from 255 to 0
                            alpha = int(255 * (feather_height - y) / feather_height)
                            if bottom - top - y - 1 >= 0:  # Ensure we're in bounds
                                mask_array[bottom - top - y - 1, :] = alpha
                    
                    # Convert back to PIL mask
                    mask = Image.fromarray(mask_array)
                    
                    # Apply mask to the flipped strip
                    result.paste(flipped_strip, (0, top), mask)
                else:
                    # No feathering, simply paste the flipped strip
                    result.paste(flipped_strip, (0, top))
        
        elif pattern_type == 'grid':
            cell_width = width // units
            cell_height = height // units
            
            for i in range(units):
                for j in range(units):
                    left = j * cell_width
                    right = (j + 1) * cell_width if j < units - 1 else width
                    top = i * cell_height
                    bottom = (i + 1) * cell_height if i < units - 1 else height
                    
                    # Crop the cell
                    cell = image.crop((left, top, right, bottom))
                    
                    # Mirror and flip the cell
                    mirrored_cell = ImageOps.mirror(cell)
                    mirrored_cell = ImageOps.flip(mirrored_cell)
                    
                    # Create feathered edges if requested
                    if use_feathering:
                        # Create a gradient mask for feathered edges
                        mask = Image.new('L', (right - left, bottom - top), 255)
                        mask_array = np.array(mask)
                        
                        # Calculate feathering dimensions (in pixels)
                        feather_width = max(1, int(cell_width * 0.1))
                        feather_height = max(1, int(cell_height * 0.1))
                        
                        # Only feather the edges that touch other cells
                        # Left edge (only if not first column)
                        if j > 0:
                            for x in range(feather_width):
                                alpha = int(255 * x / feather_width)
                                mask_array[:, x] = alpha
                        
                        # Right edge (only if not last column)
                        if j < units - 1:
                            for x in range(feather_width):
                                alpha = int(255 * (feather_width - x) / feather_width)
                                if right - left - x - 1 >= 0:
                                    mask_array[:, right - left - x - 1] = alpha
                        
                        # Top edge (only if not first row)
                        if i > 0:
                            for y in range(feather_height):
                                alpha = int(255 * y / feather_height)
                                mask_array[y, :] = alpha
                        
                        # Bottom edge (only if not last row)
                        if i < units - 1:
                            for y in range(feather_height):
                                alpha = int(255 * (feather_height - y) / feather_height)
                                if bottom - top - y - 1 >= 0:
                                    mask_array[bottom - top - y - 1, :] = alpha
                        
                        # Convert back to PIL mask
                        mask = Image.fromarray(mask_array)
                        
                        # Apply mask to the mirrored cell
                        result.paste(mirrored_cell, (left, top), mask)
                    else:
                        # No feathering, simply paste the mirrored cell
                        result.paste(mirrored_cell, (left, top))
        
        return result
    
    def create_slice_mosaic(self, image, units, x_shift, y_shift, scale_shift, edge_intensity, use_feathering, pattern_type='vertical'):
        """
        Create a slice-based mosaic effect with strips or grid.
        This mimics the look in the reference images with bands/slices and distortion.
        
        Parameters:
            image: The input PIL image
            units: Number of slices
            x_shift: Horizontal shift amount (-1.0 to 1.0)
            y_shift: Vertical shift amount (-1.0 to 1.0)
            scale_shift: Amount to scale individual slices (0.0 to 1.0)
            edge_intensity: How much stronger the effect is at the edges
            use_feathering: Whether to apply smooth transitions between slices
            pattern_type: 'vertical', 'horizontal', or 'grid'
        """
        width, height = image.size
        
        # Convert to numpy array for processing
        img_array = np.array(image)
        
        # Create arrays for the mapping coordinates
        x_coords = np.zeros((height, width), dtype=np.float32)
        y_coords = np.zeros((height, width), dtype=np.float32)
        
        # Calculate slice dimensions based on pattern type
        if pattern_type == 'vertical':
            h_units = units
            v_units = 1
            strip_width = width // h_units
            strip_height = height
        elif pattern_type == 'horizontal':
            h_units = 1
            v_units = units
            strip_width = width
            strip_height = height // v_units
        else:  # grid
            h_units = units
            v_units = units
            strip_width = width // h_units
            strip_height = height // v_units
        
        # Determine max shifts
        max_x_shift = int(strip_width * abs(x_shift) * 2)
        max_y_shift = int(strip_height * abs(y_shift) * 2)
        
        # Calculate centers
        center_x = width / 2
        center_y = height / 2
        
        # Create the mapping coordinates
        for y in range(height):
            # Calculate vertical unit index and position
            v_unit_idx = y // strip_height if v_units > 1 else 0
            unit_center_y = (v_unit_idx * strip_height) + (strip_height / 2)
            
            for x in range(width):
                # Calculate horizontal unit index and position
                h_unit_idx = x // strip_width if h_units > 1 else 0
                unit_center_x = (h_unit_idx * strip_width) + (strip_width / 2)
                
                # Calculate distance from image center (normalized for edge intensity)
                dist_from_center_x = abs(x - center_x) / center_x if center_x > 0 else 0
                dist_from_center_y = abs(y - center_y) / center_y if center_y > 0 else 0
                dist_from_center = max(dist_from_center_x, dist_from_center_y)
                
                # Scale shift by distance from center and edge intensity
                edge_factor = 1.0 + (edge_intensity - 1.0) * dist_from_center
                
                # Alternate shift direction for adjacent units
                x_direction = -1 if (h_unit_idx % 2 == 0) else 1
                y_direction = -1 if (v_unit_idx % 2 == 0) else 1
                
                # Apply the shifts
                shift_x = x_direction * edge_factor * max_x_shift * x_shift
                shift_y = y_direction * edge_factor * max_y_shift * y_shift
                
                # Apply scaling if requested (scale from unit center)
                if scale_shift > 0:
                    # Calculate scale factor (alternating larger/smaller)
                    h_scale_factor = 1.0 + ((-1)**h_unit_idx * scale_shift * 0.5)
                    v_scale_factor = 1.0 + ((-1)**v_unit_idx * scale_shift * 0.5)
                    
                    # Calculate scaled coordinates relative to unit center
                    scaled_x = unit_center_x + ((x - unit_center_x) * h_scale_factor)
                    scaled_y = unit_center_y + ((y - unit_center_y) * v_scale_factor)
                    
                    # Store mapping coordinates with both shift and scale
                    x_coords[y, x] = scaled_x + shift_x
                    y_coords[y, x] = scaled_y + shift_y
                else:
                    # Store mapping coordinates with shift only
                    x_coords[y, x] = x + shift_x
                    y_coords[y, x] = y + shift_y
        
        # Apply the remapping to create the distortion
        result_array = cv2.remap(img_array, x_coords, y_coords, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
        
        # Convert back to PIL
        result = Image.fromarray(result_array)
        
        # If feathering requested, apply targeted Gaussian blur just at the boundaries
        if use_feathering:
            # Process each unit individually with soft edges
            base = image.copy()  # Keep original image for reference
            
            # Create a full-size temporary image for the final composition
            final_result = result.copy()
            
            # Process horizontal boundaries
            if h_units > 1:
                for i in range(1, h_units):
                    boundary_x = i * strip_width
                    blur_width = max(4, int(strip_width * 0.1))  # 10% of strip width, minimum 4 pixels
                    
                    # Create a narrow vertical strip at the boundary
                    left = max(0, boundary_x - blur_width//2)
                    right = min(width, boundary_x + blur_width//2)
                    
                    # Create a mask for this boundary
                    mask = Image.new('L', (width, height), 0)
                    mask_draw = ImageDraw.Draw(mask)
                    mask_draw.rectangle([left, 0, right, height], fill=255)
                    
                    # Blur the mask for soft transitions
                    mask = mask.filter(ImageFilter.GaussianBlur(radius=blur_width//4))
                    
                    # Apply mask to blend original and distorted at the boundary
                    final_result = Image.composite(base, final_result, mask)
            
            # Process vertical boundaries
            if v_units > 1:
                for i in range(1, v_units):
                    boundary_y = i * strip_height
                    blur_height = max(4, int(strip_height * 0.1))  # 10% of strip height
                    
                    # Create a narrow horizontal strip at the boundary
                    top = max(0, boundary_y - blur_height//2)
                    bottom = min(height, boundary_y + blur_height//2)
                    
                    # Create a mask for this boundary
                    mask = Image.new('L', (width, height), 0)
                    mask_draw = ImageDraw.Draw(mask)
                    mask_draw.rectangle([0, top, width, bottom], fill=255)
                    
                    # Blur the mask for soft transitions
                    mask = mask.filter(ImageFilter.GaussianBlur(radius=blur_height//4))
                    
                    # Apply mask to blend original and distorted at the boundary
                    final_result = Image.composite(base, final_result, mask)
            
            return final_result
        
        return result

# For running directly
def run_app():
    root = tk.Tk()
    app = ImageStyleTransformer(root)
    root.mainloop()

if __name__ == "__main__":
    run_app()

2025-03-16 16:48:35.717 Python[10067:9788913] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-16 16:48:35.717 Python[10067:9788913] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 